In [1]:
#import pandas and sqlite
import pandas as pd
import sqlite3 as sl

In [2]:
#connecct to database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [3]:
#update counties table with most recent data
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE IF EXISTS counties')
c.execute('CREATE TABLE counties (date datetime, county str, state str, fips str, cases int, deaths int, confirmed_cases int, confirmed_deaths int, probable_cases int, probable_deaths int)')
conn.commit()

counties.to_sql('counties',conn, if_exists='replace')

In [4]:
#confirm tables present
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

counties


In [5]:
df = pd.read_sql('''select * from regions''',conn)
df.columns

DatabaseError: Execution failed on sql 'select * from regions': no such table: regions

In [ ]:
df = pd.read_sql('''select * from counties''',conn)
df.columns

In [ ]:
df = pd.read_sql('''WITH cte as (SELECT date, 
                                  r.bureau_economic_analysys as 'bea_region', c.state, c.county, 
                                  cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                                  cases as 'cumulative_cases'
                    FROM     counties c
                    JOIN     regions r on c.state = r.state 
                             AND r.bureau_economic_analysys = 'Great Lakes'
                    ORDER BY c.state, county)
                    
SELECT date,
       bea_region, 
       state, 
       county, 
       new_cases, 
       cumulative_cases,
       AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_new'
FROM cte
ORDER BY state, county, date''',conn)
df